In [ ]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import init

Prepare data

Util functions

In [ ]:
def init_embedding(input_embedding):
    """
    Initialize embedding
    """
    bias = np.sqrt(3.0 / input_embedding.size(1))
    nn.init.uniform(input_embedding, -bias, bias)



Build the Model



Words are modeled with **a character-based RNN with LSTM**, which produces two vectors.

The forward vector will have a representation of the character sequence from the left to the right.

The backward one will have the same in the reversed order.

Our insight is that this character-based LSTM captures the phonological structure of the word from its graphemes/characters.

These two vectors are concatenated together with the whole **word’s embedding**
(the embeddings could be pre-trained from larger corpora or trained jointly for the task).

The vector of these three elements will represent each word in the sequence. Then, for each word, there will be a **word-level LSTM**, which will produce an output for each word, with its right and left context information.

Finally, this output will go through a **CRF layer** to get the optimal output.


<img src="image_ref/figure1.png">
<img src="image_ref/tensorboard.png">

these are not showing our structure, just referrences for me.

In [ ]:
class BiLSTM_CRF(nn.Module):
    def __init__(self, vocab_size, tage_to_ix, embedding_dim, hidden_dim, char_lstm_dim=25,
                      char_to_ix=None, pre_word_embeds=None, char_embedding_dim=25, use_gpu=False,
                      n_cap=None, cap_embedding_dim=None, use_crf=True, char_mode='CNN'):
        super(BiLSTM_CRF, self).__init__()
        self.use_gpu = use_gpu
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.n_cap = n_cap
        self.cap_embedding_dim = cap_embedding_dim
        self.use_crf = use_crf
        self.tagset_size = len(tag_to_ix)
        self.out_channels = char_lstm_dim #  #output channels
        self.char_mode = char_mode
        
        print('char_mode: %s, out_channels: %d, hidden_dim: %d, ' % (char_mode, char_lstm_dim, hidden_dim))
        
        if self.n_cap and self.cap_embedding_dim:
            self.cap_embeds = nn.Embedding(self.n_cap, self.cap_embedding_dim)
            init_embedding(self.cap_embeds.weight)
        